# [Clustering4Ever](https://github.com/Clustering4Ever/Clustering4Ever) on [SparkNotebook](http://spark-notebook.io/) by [LIPN](https://lipn.univ-paris13.fr/) [A3](https://lipn.univ-paris13.fr/accueil/equipe/a3/) team

# Scala _K_-Means

In [ ]:
import smile.plot._
import scala.io.Source
import scala.collection.{immutable, mutable, parallel}
import org.clustering4ever.scala.clustering.kcenters.KMeans
import org.clustering4ever.math.distances.scalar.{Euclidean, Cosine, Minkowski}
import org.clustering4ever.scala.indexes.ExternalIndexes
import org.clustering4ever.scala.indexes.NmiNormalizationNature._
import org.clustering4ever.util.ScalaImplicits._
import org.clustering4ever.scala.clusterizables.EasyClusterizable
import org.clustering4ever.scala.vectorizables.RealVector

import smile.plot._
import scala.io.Source
import scala.collection.{immutable, mutable, parallel}
import org.clustering4ever.scala.clustering.kcenters.KMeans
import org.clustering4ever.math.distances.scalar.{Euclidean, Cosine, Minkowski}
import org.clustering4ever.scala.indexes.ExternalIndexes
import org.clustering4ever.scala.indexes.NmiNormalizationNature._
import org.clustering4ever.util.ScalaImplicits._
import org.clustering4ever.scala.clusterizables.EasyClusterizable
import org.clustering4ever.scala.vectorizables.RealVector


## Download dataset Aggregation

In [ ]:
:sh wget -P /tmp/ http://www.clustering4ever.org/Datasets/Aggregation/aggregation.csv
:sh wget -P /tmp/ http://www.clustering4ever.org/Datasets/Aggregation/labels

--2018-11-15 21:35:51--  http://www.clustering4ever.org/Datasets/Aggregation/aggregation.csv
Resolving www.clustering4ever.org (www.clustering4ever.org)... 62.210.16.62
Connecting to www.clustering4ever.org (www.clustering4ever.org)|62.210.16.62|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8063 (7.9K) [text/csv]
Saving to: ‘/tmp/aggregation.csv.2’

     0K .......                                               100% 25.7M=0s

2018-11-15 21:35:51 (25.7 MB/s) - ‘/tmp/aggregation.csv.2’ saved [8063/8063]

:sh: Scheme missing.
--2018-11-15 21:35:51--  http://wget/
Resolving wget (wget)... failed: Name or service not known.
wget: unable to resolve host address ‘wget’
--2018-11-15 21:35:51--  http://www.clustering4ever.org/Datasets/Aggregation/labels
Reusing existing connection to www.clustering4ever.org:80.
HTTP request sent, awaiting response... 200 OK
Length: 1576 (1.5K)
Saving to: ‘/tmp/labels.2’

     0K .                                                     100

## Import data as ParArray[ArrayBuffer[Double]]

In [ ]:
val datasetSize = 1000000
val dim = 3
val path = "/tmp/aggregation.csv"
val useAggregationDS = true

val rawData = if( useAggregationDS ) Source.fromFile(path).getLines.map( x => mutable.ArrayBuffer(x.split(",").map(_.toDouble):_*) ).toBuffer
  else mutable.ArrayBuffer.fill(datasetSize)(mutable.ArrayBuffer.fill(dim)(scala.util.Random.nextDouble))

val parData = rawData.zipWithIndex.par.map{ case (v, id) => new EasyClusterizable(id.toLong, new RealVector(v)) }

val groundTruePath = "/tmp/labels"

datasetSize: Int = 1000000
dim: Int = 3
path: String = /tmp/aggregation.csv
useAggregationDS: Boolean = true
rawData: scala.collection.mutable.Buffer[scala.collection.mutable.ArrayBuffer[Double]] = ArrayBuffer(ArrayBuffer(15.55, 28.65), ArrayBuffer(14.9, 27.55), ArrayBuffer(14.45, 28.35), ArrayBuffer(14.15, 28.8), ArrayBuffer(13.75, 28.05), ArrayBuffer(13.35, 28.45), ArrayBuffer(13.0, 29.15), ArrayBuffer(13.45, 27.5), ArrayBuffer(13.6, 26.5), ArrayBuffer(12.8, 27.35), ArrayBuffer(12.4, 27.85), ArrayBuffer(12.3, 28.4), ArrayBuffer(12.2, 28.65), ArrayBuffer(13.4, 25.1), ArrayBuffer(12.95, 25.95), ArrayBuffer(12.9, 26.5), ArrayBuffer(11.85, 27.0), ArrayBuffer(11.35, 28.0), ArrayBuffer(11.15, 28.7), ArrayBuffer(11.25, 27.4), ArrayBuffer(10.75, 27.7), ArrayBuffer(10.5, 28.35), ArrayBuffer(9....

## Parameters 

In [ ]:
val k = 7
val iterMax = 50
val epsilon = 0.001
// True defines the true Euclidean with square root, set it to false to avoid the square
// val metric0 = new Euclidean[Array[Double]](true)
val metric1 = new Euclidean[mutable.ArrayBuffer[Double]](false)
// Minkowski(p) where p is the Minkowski parameter
val metric2 = new Minkowski[mutable.ArrayBuffer[Double]](4)

k: Int = 7
iterMax: Int = 50
epsilon: Double = 0.001
metric1: org.clustering4ever.math.distances.scalar.Euclidean[scala.collection.mutable.ArrayBuffer[Double]] = Euclidean distance without root applied
metric2: org.clustering4ever.math.distances.scalar.Minkowski[scala.collection.mutable.ArrayBuffer[Double]] = org.clustering4ever.math.distances.scalar.Minkowski@b09fb87


## Run and measure the algorithm time

In [ ]:
val data = parData.seq

val t1 = System.nanoTime

val model = KMeans.run(parData, k, epsilon, iterMax, metric1)

val t2 = System.nanoTime

(t2 - t1) / 1000000000D

data: scala.collection.mutable.Seq[org.clustering4ever.scala.clusterizables.EasyClusterizable[Long,scala.collection.mutable.ArrayBuffer[Double],scala.collection.mutable.ArrayBuffer[Double]]] = ArraySeq(EasyClusterizable(0,RealVector(ArrayBuffer(15.55, 28.65)),None), EasyClusterizable(1,RealVector(ArrayBuffer(14.9, 27.55)),None), EasyClusterizable(2,RealVector(ArrayBuffer(14.45, 28.35)),None), EasyClusterizable(3,RealVector(ArrayBuffer(14.15, 28.8)),None), EasyClusterizable(4,RealVector(ArrayBuffer(13.75, 28.05)),None), EasyClusterizable(5,RealVector(ArrayBuffer(13.35, 28.45)),None), EasyClusterizable(6,RealVector(ArrayBuffer(13.0, 29.15)),None), EasyClusterizable(7,RealVector(ArrayBuffer(13.45, 27.5)),None), EasyClusterizable(8,RealVector(ArrayBuffer(13.6, 26.5)),None), EasyClusterizabl...

0.54154205

# Compare algorithm between ParArray and Seq container

In [ ]:
val t1 = System.nanoTime
KMeans.run(parData, k, epsilon, iterMax, metric1)
val t2 = System.nanoTime
val seqData = parData.seq
val t3 = System.nanoTime
KMeans.run(seqData, k, epsilon, iterMax, metric1)
val t4 = System.nanoTime



(t4 - t3).toDouble / (t2 - t1)

t1: Long = 1569027586396
t2: Long = 1569088191256
seqData: scala.collection.mutable.Seq[org.clustering4ever.scala.clusterizables.EasyClusterizable[Long,scala.collection.mutable.ArrayBuffer[Double],scala.collection.mutable.ArrayBuffer[Double]]] = ArraySeq(EasyClusterizable(0,RealVector(ArrayBuffer(15.55, 28.65)),None), EasyClusterizable(1,RealVector(ArrayBuffer(14.9, 27.55)),None), EasyClusterizable(2,RealVector(ArrayBuffer(14.45, 28.35)),None), EasyClusterizable(3,RealVector(ArrayBuffer(14.15, 28.8)),None), EasyClusterizable(4,RealVector(ArrayBuffer(13.75, 28.05)),None), EasyClusterizable(5,RealVector(ArrayBuffer(13.35, 28.45)),None), EasyClusterizable(6,RealVector(ArrayBuffer(13.0, 29.15)),None), EasyClusterizable(7,RealVector(ArrayBuffer(13.45, 27.5)),None), EasyClusterizable(8,RealVe...

0.1441250751177381

## Different ways to apply model to datasets

In [ ]:
val clusterized11 = rawData.map( v => (model.centerPredict(v), v) )
val clusterized12 = model.centerPredict(rawData)

val clusterized21 = parData.map( clustbl => (model.centerPredict(clustbl.vector), clustbl) )
val clusterized22 = model.centerPredict(parData)

clusterized11: scala.collection.mutable.Buffer[(model.ClusterID, scala.collection.mutable.ArrayBuffer[Double])] = ArrayBuffer((5,ArrayBuffer(15.55, 28.65)), (0,ArrayBuffer(14.9, 27.55)), (0,ArrayBuffer(14.45, 28.35)), (0,ArrayBuffer(14.15, 28.8)), (0,ArrayBuffer(13.75, 28.05)), (0,ArrayBuffer(13.35, 28.45)), (0,ArrayBuffer(13.0, 29.15)), (0,ArrayBuffer(13.45, 27.5)), (0,ArrayBuffer(13.6, 26.5)), (0,ArrayBuffer(12.8, 27.35)), (0,ArrayBuffer(12.4, 27.85)), (0,ArrayBuffer(12.3, 28.4)), (0,ArrayBuffer(12.2, 28.65)), (0,ArrayBuffer(13.4, 25.1)), (0,ArrayBuffer(12.95, 25.95)), (0,ArrayBuffer(12.9, 26.5)), (0,ArrayBuffer(11.85, 27.0)), (0,ArrayBuffer(11.35, 28.0)), (0,ArrayBuffer(11.15, 28.7)), (0,ArrayBuffer(11.25, 27.4)), (0,ArrayBuffer(10.75, 27.7)), (0,ArrayBuffer(10.5, 28.35)), (0,ArrayBu...

## Plot clustering results

In [ ]:
val rawData = clusterized22.map( rc => rc.vector.toArray ).toArray
val labels = clusterized22.map( rc => rc.clusterID.get ).toArray
plot(rawData, labels, '*', Palette.COLORS)

rawData: Array[Array[Double]] = Array(Array(15.55, 28.65), Array(14.9, 27.55), Array(14.45, 28.35), Array(14.15, 28.8), Array(13.75, 28.05), Array(13.35, 28.45), Array(13.0, 29.15), Array(13.45, 27.5), Array(13.6, 26.5), Array(12.8, 27.35), Array(12.4, 27.85), Array(12.3, 28.4), Array(12.2, 28.65), Array(13.4, 25.1), Array(12.95, 25.95), Array(12.9, 26.5), Array(11.85, 27.0), Array(11.35, 28.0), Array(11.15, 28.7), Array(11.25, 27.4), Array(10.75, 27.7), Array(10.5, 28.35), Array(9.65, 28.45), Array(10.25, 27.25), Array(10.75, 26.55), Array(11.7, 26.35), Array(11.6, 25.9), Array(11.9, 25.05), Array(12.6, 24.05), Array(11.9, 24.5), Array(11.1, 25.2), Array(10.55, 25.15), Array(10.05, 25.95), Array(9.35, 26.6), Array(9.3, 27.25), Array(9.2, 27.8), Array(7.5, 28.25), Array(8.55, 27.45), Ar...

Window(javax.swing.JFrame[frame0,460,37,1000x1000,layout=java.awt.BorderLayout,title=Smile Plot 1,resizable,normal,defaultCloseOperation=DISPOSE_ON_CLOSE,rootPane=javax.swing.JRootPane[,0,37,1000x963,layout=javax.swing.JRootPane$RootLayout,alignmentX=0.0,alignmentY=0.0,border=,flags=16777675,maximumSize=,minimumSize=,preferredSize=],rootPaneCheckingEnabled=true],smile.plot.PlotCanvas[,0,0,1000x963,layout=java.awt.BorderLayout,alignmentX=0.0,alignmentY=0.0,border=,flags=11,maximumSize=,minimumSize=,preferredSize=])

## Inspect performance metrics

In [ ]:
val trueLabels = Source.fromFile(groundTruePath).getLines.map(_.toInt).toBuffer.toParArray

val (_, trueLabelsFrom0) = ExternalIndexes.prepareLabels(trueLabels)

val predicts = clusterized22.map(_.clusterID.get).toParArray

val nmi = ExternalIndexes.nmi(trueLabelsFrom0, predicts, SQRT)

nmi

trueLabels: scala.collection.parallel.mutable.ParArray[Int] = ParArray(2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, ...

0.851792189190695